In [1]:
from random import randint, uniform
from util import generate_problem, verify_witness

In [4]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def sweep_line(intervals):
    L = []
    for l, r in intervals:
        L.append( (l, 1) )
        L.append( (r, -1) )
        
    L.sort(key=lambda a: 10*a[0] - a[1])
    
    c = 0
    top = -1
    x = 0
    
    for p, i in L:
        c += i
        if c > top:
            top = c
            x = p
    
    # returns the number of constraints unsatisfied and time point
    return len(intervals) - top, x

def walk_gene(gene, T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    
    constraints_per_variable = [list() for i in range(num_variables + 1)]
    # at index i should contain availability intervals (given other variables are fixed.)
    # remark: existence of an interval is local with respect to one constraint
    # remark: so if intersection is empty, then there is no "safe" place to put this variable
    
    for c in T:
        i, j = c['i'], c['j']
        intervals = c['intervals']
        for l, r in intervals:
            # X_j - X_i in (l, r)
            # fix X_j to genes[j]
            # so genes[j] - X_i in (l, r)
            # then X_i in [X_j - r, X_j - l]
            constraints_per_variable[i].append( (gene[j] - r, gene[j] - l) )
            
            # now for j:
            constraints_per_variable[j].append( (gene[i] + l, gene[i] + r) )
            
            # ^ tested this on paper, it's ok.
    
    
    # for each constraint, sweep line, find best option
    # (later? we want to pick whatever REDUCES the unsatisfieds the most)
    # for now: pick a random variable and modify it according to sweep_line
    # if no later, then we can make the loop upstairs simpler (to only do stuff if "i" is mentioned)
    i = randint(1, num_variables)
    gene[i] = sweep_line(constraints_per_variable[i])[1]

    return gene
    
def fitness(gene, T):
    # TODO
    # different ways of computing fitness
    # like: max distance for constraint to be satisfied
    #       number of unsatisfied constraints
    #       this, but weighted somehow, etc
    
    return -len(verify_witness(gene, T))

def select(genes, retainment_ratio, T):
    genes.sort(key=lambda g: -fitness(g, T))
    return genes[: int(len(genes)*retainment_ratio+1) ]

def crossover(genes, gene_pool_size):
    while len(genes) < gene_pool_size:
        i = randint(0, len(genes)-1)
        j = randint(0, len(genes)-1)
        cross_index = randint(1, gene_pool_size-2)
        genes += [ genes[i][cross_index:] + genes[j][:cross_index] ]
        
    return genes

def mutate(T, genes, mutation_chance, r):
    for g in genes:
        if uniform(0, 1) < mutation_chance:
    # TODO:
    #        g[randint(0, len(g)-1)] = randint(-r, r)
    # or
            walk_gene(g, T)
            
    return genes
    
def genetic(T, r,
            gene_pool_size,
            retainment_ratio,
            mutation_chance,
            max_iterations):
    
    genes = [random_gene(T, r) for i in range(gene_pool_size)]
    
    for it in range(max_iterations):
        genes = select(genes, retainment_ratio, T)
        genes = crossover(genes, gene_pool_size)
        genes = mutate(T, genes, mutation_chance, r)
    
    best_gene = select(genes, 1, T)[0] # to sort genes such that first is best
    best_gene_failed = verify_witness(best_gene, T)
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [39]:
T = generate_problem(
    20,
    min_intervals=1,
    max_intervals=1,
    scaling_factor=1
)

genetic(
    T=T, r=100,
    gene_pool_size=50,
    retainment_ratio=0.1,
    mutation_chance=0.1,
    max_iterations=50,
)

best gene: [0, -77, 10, -13, -83, -16, -21, 62, -79, -16, -100, -89, -100, -93, -87, -36, -100, -75, -54, -53]
constraints failed: 7 out of: 57
failed constraints: [{'i': 1, 'j': 7, 'intervals': [(37, 80)]}, {'i': 1, 'j': 13, 'intervals': [(-43, -35)]}, {'i': 1, 'j': 16, 'intervals': [(5, 81)]}, {'i': 1, 'j': 17, 'intervals': [(63, 64)]}, {'i': 2, 'j': 15, 'intervals': [(5, 32)]}, {'i': 8, 'j': 18, 'intervals': [(-87, -23)]}, {'i': 9, 'j': 15, 'intervals': [(-87, -57)]}]
